In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

repo_root = Path.cwd().parent  # docs/ -> repo root
sys.path.insert(0, str(repo_root))
# import tests.fixtures.test_case_catalog as cases
import tests.fixtures.test_case_catalog as cases
import tests.utils_for_testing as tst_ut


# single sim test case

In [2]:
case = cases.Local_TestCases.retrieve_norfolk_all_models_test_case(start_from_scratch=True)
analysis = case.analysis
system = analysis._system

## compilation

In [3]:
# SWMM
system.compile_SWMM(
    recompile_if_already_done_successfully=False,
    # redownload_swmm_if_exists=True,
)
# TRITON
system.compile_TRITON_only(recompile_if_already_done_successfully=False)
system.compile_TRITON_SWMM(recompile_if_already_done_successfully=False)


[SWMM] Already compiled successfully (skipping)

Compiling TRITON-only CPU Backend
[TRITON-only CPU] Already compiled successfully (skipping)

Compiling CPU Backend
[CPU] Already compiled successfully (skipping)


## scenario prep

In [4]:
analysis.run_prepare_scenarios_serially(
    overwrite_scenarios=True, rerun_swmm_hydro_if_outputs_exist=True
)
tst_ut.assert_scenarios_setup(analysis)

2026-02-05 17:13:49,378 [INFO] Loading system configuration from /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/tests.yaml
2026-02-05 17:13:49,381 [INFO] Loading analysis configuration from /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/all_models/all_models.yaml
2026-02-05 17:13:49,388 [INFO] Preparing scenario 0
[Scenario 0] Using CPU backend

... SWMM Version 5.2.4
 o  Retrieving project data
 o  Simulating day: 0     hour: Simulation complete           
... Run CompleteScenario preparation complete
2026-02-05 17:13:52,625 [INFO] Scenario 0 prepared successfully


## run

In [5]:
analysis.run_sims_in_sequence(pickup_where_leftoff=False)
tst_ut.assert_scenarios_run(analysis)
for model_type in tst_ut.get_enabled_model_types(analysis):
    tst_ut.assert_model_simulation_run(analysis, model_type)

/home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/all_models/sims/0-event_id.0


Failed: One or more simulations failed to run

In [11]:
case.analysis.process_all_sim_timeseries_serially()

Processing run results for scenario 0
Processed TRITON outputs for scenario 0
##################################
Found problem. Orifice conduits do not return max velocity or max over full flow. Filling with empty string
Normal row vs. problem row:
  0                    CONDUIT     0.000     0  00:01      0.00    0.00    0.00

  756                  ORIFICE     0.000     0  00:00                      0.00

Properly parsed values:
['756', 'ORIFICE', '0.000', '0', '00:00', '', '', '0.00\n']
Converted variable to datatype = type, <class 'str'>
Converted variable to datatype = type, <class 'str'>
Processed SWMM outputs for scenario 0


# multi-sim case

In [ ]:
case = cases.Local_TestCases.retrieve_norfolk_multi_sim_test_case(start_from_scratch=False)
analysis = case.analysis

In [13]:
# analysis = norfolk_multi_sim_analysis

result = analysis.submit_workflow(
    mode="local",
    process_system_level_inputs=True,
    overwrite_system_inputs=True,
    compile_TRITON_SWMM=True,
    recompile_if_already_done_successfully=True,
    prepare_scenarios=True,
    overwrite_scenario=True,
    rerun_swmm_hydro_if_outputs_exist=True,
    process_timeseries=True,
    which="both",
    clear_raw_outputs=True,
    overwrite_if_exist=True,
    compression_level=5,
    pickup_where_leftoff=False,
    dry_run=False,
    verbose=True,
)

[Snakemake] Submitting workflow in local mode
[Snakemake] Snakefile generated: /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/multi_sim/Snakefile
[Snakemake] Running workflow locally with Snakemake
[Snakemake] DRY RUN
[Snakemake] Using dynamic config from: /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/multi_sim/.snakemake_profile/local


[Snakemake] Workflow completed successfully
[Snakemake] Running workflow locally with Snakemake
[Snakemake] Using dynamic config from: /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/multi_sim/.snakemake_profile/local
[Snakemake] Workflow completed successfully


In [9]:
analysis.SWMM_node_summary

ValueError: could not open file because it does not exist: /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/multi_sim/SWMM_nodes.zarr. Run analysis.consolidate_[SWMM/TRITON]_outputs().

# sensitivity analysis

In [ ]:
nrflk_cpu_sensitivity = cases.Local_TestCases.retrieve_norfolk_cpu_config_sensitivity_case(
        start_from_scratch=True
    )
analysis = nrflk_cpu_sensitivity.analysis

In [6]:
result = analysis.submit_workflow(
    mode="local",
    process_system_level_inputs=True,
    overwrite_system_inputs=True,
    compile_TRITON_SWMM=True,
    recompile_if_already_done_successfully=True,
    prepare_scenarios=True,
    overwrite_scenario=True,
    rerun_swmm_hydro_if_outputs_exist=True,
    process_timeseries=True,
    which="both",
    clear_raw_outputs=True,
    overwrite_if_exist=True,
    compression_level=5,
    pickup_where_leftoff=False,
    dry_run=True,
    verbose=True,
)

[Snakemake] Submitting sensitivity analysis workflow in local mode
[Snakemake] Generated master Snakefile: /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/cpu_config_sensitivity/Snakefile
[Snakemake] Created required directories (_status, logs/sims)
[Snakemake] Running workflow locally with Snakemake
[Snakemake] Using dynamic config from: /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/cpu_config_sensitivity/.snakemake_profile/local
[Snakemake] Workflow completed successfully


In [ ]:
case = cases.Local_TestCases.retrieve_norfolk_cpu_config_sensitivity_case(start_from_scratch=False)
analysis = case.analysis

In [ ]:
analysis.sensitivity.compile_TRITON_SWMM_for_sensitivity_analysis()
assert analysis.sensitivity.compilation_successful == True

In [ ]:
analysis.sensitivity.prepare_scenarios_in_each_subanalysis(concurrent=True)
assert analysis.log.all_scenarios_created.get() == True

In [ ]:
analysis.sensitivity.run_all_sims(pickup_where_leftoff=False, concurrent=False)
# assert analysis.log.all_sims_run.get() == True